In [3]:
!git clone -b prune-experiment https://github.com/atikul-islam-sajib/TreeBasedModel.git
# git clone -b prune-experiment https://github.com/markusloecher/TreeSandBox.git

Cloning into 'TreeBasedModel'...
remote: Enumerating objects: 988, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 988 (delta 48), reused 44 (delta 18), pack-reused 907
Receiving objects: 100% (988/988), 124.17 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (550/550), done.


In [7]:
%cd /content/TreeBasedModel

/content/TreeBasedModel


In [8]:
!pip install -e . --verbose

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/TreeBasedModel
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info
  writing /tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-jco96fui/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  writing manifest file '/tmp/pip-pip-e

In [9]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest
from copy import deepcopy

In [25]:
# Generate synthetic dataset
X, y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_clusters_per_class=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Instantiate RandomForest
rf = RandomForest(n_trees=2, max_depth=10, min_samples_split=20, min_samples_leaf=5,
                  n_feature="sqrt", bootstrap=True, oob=True, criterion="gini",
                  treetype="classification", random_state=42)

# Fit the model
rf.fit(X_train, y_train)


/content/TreeBasedModel/TreeModelsFromScratch/RandomForest.py:277: UserWarning: 25 out of 75 samples do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates. These samples were dropped before computing the oob_score
  warn(message)


In [26]:
X_train.shape

(75, 20)

Understand the tree structure first

In [27]:
dt =deepcopy(rf.trees[0])
rootNode = dt.node_list[0] #??
print(rootNode.samples)
print(rootNode.threshold)
print(rootNode.value)
rootNode.left.samples
rootNode.right.leaf_node
rootNode.right.samples
dt._get_y_for_node(rootNode.right)
rootNode.right.id
rootNode.left.id
dt.decision_paths


75
-0.6577327560706901
0


[(0, 1), (0, 2, 3), (0, 2, 4, 5), (0, 2, 4, 6)]

## Try a recursive function

In [28]:
def traverseTree(parentNode, numLeafs):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return numLeafs+1
    else:
        print("inner node", parentNode.id, "with", parentNode.samples, "samples")
        if parentNode.left != None:
            print("left turn")
            numLeafs = traverseTree(parentNode.left,numLeafs)
        if parentNode.right != None:
            print("right turn")
            numLeafs = traverseTree(parentNode.right, numLeafs)
        return numLeafs

nLeafs = traverseTree(rootNode,0)
print(nLeafs, "leaves in total")

inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
inner node 2 with 55 samples
left turn
leaf node 3 with 17 samples
right turn
inner node 4 with 38 samples
left turn
leaf node 5 with 5 samples
right turn
leaf node 6 with 33 samples
4 leaves in total


Now we use the same idea of a recursive function to prune:

In [29]:
import math

def pruneTree(parentNode, min_samples_leaf = 10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right
        print("inner node", parentNode.id, "with", parentNode.samples, "samples and children:",leftChild.id, rightChild.id)
        if (leftChild.samples < min_samples_leaf) & (rightChild.samples < min_samples_leaf):#easiest case
            print('\033[1m' + "pruning both children", leftChild.id, "with", leftChild.samples, "samples", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
            parentNode.leaf_node = True
            parentNode.left = None
            parentNode.right = None
        elif (leftChild.samples >= min_samples_leaf) & (rightChild.samples >= min_samples_leaf):#also easy
            print("left turn to ", leftChild.id)
            pruneTree(leftChild,min_samples_leaf)
            print("right turn to ", rightChild.id)
            pruneTree(rightChild,min_samples_leaf)
        elif leftChild.samples < min_samples_leaf:
            print('\033[1m' + "pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
            parentNode.left = None
            #reminder: left_idxs = np.argwhere(X_column <= split_thresh).flatten()
            parentNode.threshold = -math.inf#ideally one should remove this useless inner node
            print("right turn to ", rightChild.id)
            pruneTree(rightChild,min_samples_leaf)
        elif rightChild.samples < min_samples_leaf:
            print('\033[1m' + "pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
            parentNode.right = None
            parentNode.threshold = math.inf#ideally one should remove this useless inner node
            print("left turn to ", leftChild.id)
            pruneTree(leftChild,min_samples_leaf)

    #print("parent node", parentNode.id, "with", "children:",parentNode.left, parentNode.right)
    #if (parentNode.left == None) & (parentNode.right == None):
    #    parentNode.leaf_node = True

dt =deepcopy(rf.trees[0])
rootNode = dt.node_list[0] #??

pruneTree(rootNode,20)
print("------------ done pruning, now just traversing again:---------------")
nLeafs = traverseTree(rootNode,0)
print(nLeafs, "leaves in total")

inner node 0 with 75 samples and children: 1 2
left turn to  1
leaf node 1 with 20 samples
right turn to  2
inner node 2 with 55 samples and children: 3 4
pruning left child 3 with 17 samples
right turn to  4
inner node 4 with 38 samples and children: 5 6
pruning left child 5 with 5 samples
right turn to  6
leaf node 6 with 33 samples
------------ done pruning, now just traversing again:---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
inner node 2 with 55 samples
right turn
inner node 4 with 38 samples
right turn
leaf node 6 with 33 samples
2 leaves in total


While the routine above sort of works, it needs two major improvements:
1. The samples in the pruned nodes are discarded at the moment, whereas I think it would be better to redistribute them along all the children.
2. When only one child is pruned, the other surviving one becomes a useless inner node without a split. (I set the split threshold to infinity) So best would be to remove that node.

# **Task 1 & 2**

**Task 1**: Redistribute Samples from Pruned Nodes to All Remaining Children
1. **Function Creation:** Developed a function `redistribute_samples_to_children` to distribute samples from pruned nodes to their remaining sibling nodes.

2. **Integration with Pruning Logic:** Incorporated this function into the pruning process so that whenever a node is pruned, its samples are redistributed among the remaining children.

**Task 2**: Remove Useless Inner Nodes and Promote the Surviving Child
1. **Identifying Useless Inner Nodes:** Updated the pruning logic to detect when only one child remains after pruning.

2. **Promotion of Surviving Child:** Removed the useless inner node by promoting the surviving child, transferring its properties to the parent node.

3. **Recursive Pruning:** Ensured that the pruning process continues recursively if the promoted node is not a leaf node.

In [30]:
import math

def redistribute_samples_to_children(parentNode, prunedChild):
    remaining_children = [parentNode.left, parentNode.right]
    remaining_children = [child for child in remaining_children if child and child.id != prunedChild.id]

    if prunedChild and prunedChild.leaf_node and remaining_children:
        num_remaining_children = len(remaining_children)
        samples_per_child = prunedChild.samples // num_remaining_children
        for child in remaining_children:
            print(f"Redistributing {samples_per_child} samples from pruned node {prunedChild.id} to child node {child.id}")
            child.samples += samples_per_child

def pruneTree(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right
        print("inner node", parentNode.id, "with", parentNode.samples, "samples and children:", leftChild.id if leftChild else None, rightChild.id if rightChild else None)

        if leftChild and rightChild:
            if (leftChild.samples < min_samples_leaf) and (rightChild.samples < min_samples_leaf):
                # Prune both children
                print('\033[1m' + "Pruning both children", leftChild.id, "with", leftChild.samples, "samples", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.leaf_node = True
                parentNode.left = None
                parentNode.right = None
                # Distribute samples from pruned nodes
                #redistribute_samples_to_children(parentNode, leftChild)
                #redistribute_samples_to_children(parentNode, rightChild)
            elif (leftChild.samples >= min_samples_leaf) and (rightChild.samples >= min_samples_leaf):
                # Recurse on both children
                print("left turn to ", leftChild.id)
                pruneTree(leftChild, min_samples_leaf)
                print("right turn to ", rightChild.id)
                pruneTree(rightChild, min_samples_leaf)
            elif leftChild.samples < min_samples_leaf:
                # Prune left child only
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, leftChild)
                parentNode.left = None
                print(f"Removing useless inner node {parentNode.id} and promoting right child {rightChild.id}")
                # Remove useless inner node and promote the right child
                parentNode.left = rightChild.left
                parentNode.right = rightChild.right
                parentNode.threshold = rightChild.threshold
                parentNode.leaf_node = rightChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)
            elif rightChild.samples < min_samples_leaf:
                # Prune right child only
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, rightChild)
                parentNode.right = None
                print(f"Removing useless inner node {parentNode.id} and promoting left child {leftChild.id}")
                # Remove useless inner node and promote the left child
                parentNode.left = leftChild.left
                parentNode.right = leftChild.right
                parentNode.threshold = leftChild.threshold
                parentNode.leaf_node = leftChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)

        elif leftChild:
            if leftChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                parentNode.left = None
                #redistribute_samples_to_children(parentNode, leftChild)
                parentNode.leaf_node = True

        elif rightChild:
            if rightChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.right = None
                #redistribute_samples_to_children(parentNode, rightChild)
                parentNode.leaf_node = True

if __name__ == "__main__":
  dt = deepcopy(rf.trees[0])
  rootNode = dt.node_list[0]

  pruneTree(rootNode, 20)
  print("------------ done pruning, now just traversing again:---------------")
  nLeafs = traverseTree(rootNode, 0)
  print(nLeafs, "leaves in total")

inner node 0 with 75 samples and children: 1 2
left turn to  1
leaf node 1 with 20 samples
right turn to  2
inner node 2 with 55 samples and children: 3 4
Pruning left child 3 with 17 samples
Redistributing 17 samples from pruned node 3 to child node 4
Removing useless inner node 2 and promoting right child 4
inner node 2 with 55 samples and children: 5 6
Pruning left child 5 with 5 samples
Redistributing 5 samples from pruned node 5 to child node 6
Removing useless inner node 2 and promoting right child 6
------------ done pruning, now just traversing again:---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
leaf node 2 with 55 samples
2 leaves in total


## Refining Redistribution of samples to child nodes

The current function `redistribute_samples_to_children` redistributes the samples only to the next child, not all children down the path. That works well for the given example (as the remaining children are pruned anyways) but  unfortunately it is not sufficient to only pass to the immediate child.

The case where no more children further downstream would be pruned is an example for such a use case.

We will have to write a function which updates all the children downwards from the respective node. This would be a modification of the existing function `_reestimate_node_values()` which always starts at the root node. Can we modify it so that it starts at any node and then just re-estimates the node values and sample counts going down the tree from that node require a traversal of the subtree.

The rough idea would be as follows:
When we prune the "left child 3 with 17 samples" we currently redistribute 17 samples from pruned node 3 to child node 4. But we have to go deeper.
(The code below assumes -incorrectly I think- that the data belonging to node3 are just from X_train)

In [31]:
dir(dt)
node3= dt.node_id_dict[3]["node"]
node4= dt.node_id_dict[4]["node"]
#dt.node_id_dict[3]
#dir(node3)#.samples
node3Samples = np.array(X_train.iloc[node3.sample_indices,:])
print(node3Samples.shape)
node3Samples
np.array([dt.traverse_explain_path(x, node4) for x in node3Samples], dtype="object")
#dt.traverse_explain_path(x=node3Samples,node=node4)
#dt.traverse_explain_path(x=node3Samples,node=rootNode)
#rootNode.sample_indices
#dir(rf)
#rf._bootstrap_samples()
#rf.bootstrap

(17, 20)


array([[[4, 6],
        [{'node_id': 4, 'feature': 17, 'threshold': -2.862, 'value_observation': -1.428, 'decision': '-1.428 > -2.862 --> right'},
         {'node_id': 6, 'value': 0, 'prob_distribution': array([1., 0.])}]],

       [[4, 6],
        [{'node_id': 4, 'feature': 17, 'threshold': -2.862, 'value_observation': -2.152, 'decision': '-2.152 > -2.862 --> right'},
         {'node_id': 6, 'value': 0, 'prob_distribution': array([1., 0.])}]],

       [[4, 6],
        [{'node_id': 4, 'feature': 17, 'threshold': -2.862, 'value_observation': 3.359, 'decision': '3.359 > -2.862 --> right'},
         {'node_id': 6, 'value': 0, 'prob_distribution': array([1., 0.])}]],

       [[4, 5],
        [{'node_id': 4, 'feature': 17, 'threshold': -2.862, 'value_observation': -5.523, 'decision': '-5.523 <= -2.862 --> left'},
         {'node_id': 5, 'value': 0, 'prob_distribution': array([0.8, 0.2])}]],

       [[4, 6],
        [{'node_id': 4, 'feature': 17, 'threshold': -2.862, 'value_observation': 1.8

In [32]:
traversed_nodes = dt.explain_decision_path(node3Samples)#[:, 0].copy()
traversed_nodes.shape#[0]

(17, 2, 2)

In [33]:
dt._reestimate_node_values(node3Samples,)

TypeError: DecisionTree._reestimate_node_values() missing 1 required positional argument: 'y'

In [34]:
import math
from copy import deepcopy

def redistribute_samples_to_children(parentNode, prunedChild):
    remaining_children = [parentNode.left, parentNode.right]
    remaining_children = [child for child in remaining_children if child and child.id != prunedChild.id]

    if prunedChild and prunedChild.leaf_node and remaining_children:
        num_remaining_children = len(remaining_children)
        samples_per_child = prunedChild.samples // num_remaining_children
        for child in remaining_children:
            print(f"Redistributing {samples_per_child} samples from pruned node {prunedChild.id} to child node {child.id}")
            child.samples += samples_per_child

def pruneTree(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right
        print("inner node", parentNode.id, "with", parentNode.samples, "samples and children:", leftChild.id if leftChild else None, rightChild.id if rightChild else None)

        if leftChild and rightChild:
            if (leftChild.samples < min_samples_leaf) and (rightChild.samples < min_samples_leaf):
                # Prune both children
                print('\033[1m' + "Pruning both children", leftChild.id, "with", leftChild.samples, "samples", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.leaf_node = True
                parentNode.left = None
                parentNode.right = None
            elif (leftChild.samples >= min_samples_leaf) and (rightChild.samples >= min_samples_leaf):
                # Recurse on both children
                print("left turn to ", leftChild.id)
                pruneTree(leftChild, min_samples_leaf)
                print("right turn to ", rightChild.id)
                pruneTree(rightChild, min_samples_leaf)
            elif leftChild.samples < min_samples_leaf:
                # Prune left child only
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, leftChild)
                parentNode.left = None
                print(f"Removing useless inner node {parentNode.id} and promoting right child {rightChild.id}")
                # Remove useless inner node and promote the right child
                parentNode.left = rightChild.left
                parentNode.right = rightChild.right
                parentNode.threshold = rightChild.threshold
                parentNode.leaf_node = rightChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)
            elif rightChild.samples < min_samples_leaf:
                # Prune right child only
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                redistribute_samples_to_children(parentNode, rightChild)
                parentNode.right = None
                print(f"Removing useless inner node {parentNode.id} and promoting left child {leftChild.id}")
                # Remove useless inner node and promote the left child
                parentNode.left = leftChild.left
                parentNode.right = leftChild.right
                parentNode.threshold = leftChild.threshold
                parentNode.leaf_node = leftChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)

        elif leftChild:
            if leftChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning left child", leftChild.id, "with", leftChild.samples, "samples" + '\033[0m')
                parentNode.left = None
                parentNode.leaf_node = True

        elif rightChild:
            if rightChild.samples < min_samples_leaf:
                print('\033[1m' + "Pruning right child", rightChild.id, "with", rightChild.samples, "samples" + '\033[0m')
                parentNode.right = None
                parentNode.leaf_node = True

def reestimate_node_values(node):
    if node.leaf_node:
        # Base case: If it's a leaf node, just return its sample count
        return node.samples

    # Initialize the sample count for the current node
    node.samples = 0

    # Re-estimate for the left child if it exists
    if node.left:
        node.samples += reestimate_node_values(node.left)

    # Re-estimate for the right child if it exists
    if node.right:
        node.samples += reestimate_node_values(node.right)

    # Recalculate any other values at this node, such as class distributions or thresholds
    # This will depend on how the tree is implemented and what values are stored at each node
    # For instance, you might want to re-calculate the Gini impurity or any other criteria used

    return node.samples  # Return the updated sample count for this node

# Function to start re-estimation from any node
def reestimate_subtree_values(starting_node):
    if starting_node:
        reestimate_node_values(starting_node)

def traverseTree(parentNode, numLeafs):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", parentNode.samples, "samples")
        return numLeafs+1
    else:
        print("inner node", parentNode.id, "with", parentNode.samples, "samples")
        if parentNode.left is not None:
            print("left turn")
            numLeafs = traverseTree(parentNode.left, numLeafs)
        if parentNode.right is not None:
            print("right turn")
            numLeafs = traverseTree(parentNode.right, numLeafs)
        return numLeafs

if __name__ == "__main__":
    # Assuming rf is the RandomForest instance and trees is a list of decision trees within it
    dt = deepcopy(rf.trees[0])
    rootNode = dt.node_list[0]

    print("------------ before pruning: ---------------")
    nLeafs = traverseTree(rootNode, 0)
    print(nLeafs, "leaves in total")

    pruneTree(rootNode, 20)

    print("------------ done pruning, now just traversing again: ---------------")
    reestimate_subtree_values(rootNode)
    nLeafs = traverseTree(rootNode, 0)
    print(nLeafs, "leaves in total")

------------ before pruning: ---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
inner node 2 with 55 samples
left turn
leaf node 3 with 17 samples
right turn
inner node 4 with 38 samples
left turn
leaf node 5 with 5 samples
right turn
leaf node 6 with 33 samples
4 leaves in total
inner node 0 with 75 samples and children: 1 2
left turn to  1
leaf node 1 with 20 samples
right turn to  2
inner node 2 with 55 samples and children: 3 4
Pruning left child 3 with 17 samples
Redistributing 17 samples from pruned node 3 to child node 4
Removing useless inner node 2 and promoting right child 4
inner node 2 with 55 samples and children: 5 6
Pruning left child 5 with 5 samples
Redistributing 5 samples from pruned node 5 to child node 6
Removing useless inner node 2 and promoting right child 6
------------ done pruning, now just traversing again: ---------------
inner node 0 with 75 samples
left turn
leaf node 1 with 20 samples
right turn
leaf node 2 with

In [37]:
def redistribute_samples_downwards(node, additional_samples):
    if not node:
        return 0  # Return if node is None

    # Add the additional samples to the current node
    node.samples += additional_samples

    # Traverse and update left child if exists
    if node.left:
        redistribute_samples_downwards(node.left, additional_samples)

    # Traverse and update right child if exists
    if node.right:
        redistribute_samples_downwards(node.right, additional_samples)

def prune_and_redistribute(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        return  # Base case: If it's a leaf node, do nothing
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right

        # Check for children's samples
        if leftChild and leftChild.samples < min_samples_leaf:
            # Prune left child and redistribute its samples
            print(f"Pruning left child {leftChild.id} with {leftChild.samples} samples")
            redistribute_samples_downwards(parentNode, leftChild.samples)
            parentNode.left = None  # Remove the pruned node

        if rightChild and rightChild.samples < min_samples_leaf:
            # Prune right child and redistribute its samples
            print(f"Pruning right child {rightChild.id} with {rightChild.samples} samples")
            redistribute_samples_downwards(parentNode, rightChild.samples)
            parentNode.right = None  # Remove the pruned node

        # Recurse on any children that are still present
        if parentNode.left:
            prune_and_redistribute(parentNode.left, min_samples_leaf)
        if parentNode.right:
            prune_and_redistribute(parentNode.right, min_samples_leaf)

        # Update current node to leaf if both children are pruned
        if parentNode.left is None and parentNode.right is None:
            parentNode.leaf_node = True

In [43]:
# Traverse tree function
def traverseTree(parentNode, numLeafs):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", len(parentNode.samples), "samples")
        return numLeafs + 1
    else:
        print("inner node", parentNode.id, "with", len(parentNode.samples), "samples")
        if parentNode.left is not None:
            print("left turn")
            numLeafs = traverseTree(parentNode.left, numLeafs)
        if parentNode.right is not None:
            print("right turn")
            numLeafs = traverseTree(parentNode.right, numLeafs)
        return numLeafs

# Prune tree function
def pruneTree(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        print("leaf node", parentNode.id, "with", len(parentNode.samples), "samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right

        if not isinstance(parentNode.samples, list):
            raise ValueError(f"Node {parentNode.id} has samples as an integer, expected a list of samples.")

        # Ensure children nodes' samples are lists
        if leftChild is not None and isinstance(leftChild.samples, int):
            leftChild.samples = [leftChild.samples]
        if rightChild is not None and isinstance(rightChild.samples, int):
            rightChild.samples = [rightChild.samples]

        print("inner node", parentNode.id, "with", len(parentNode.samples), "samples and children:", leftChild.id, rightChild.id)

        if len(leftChild.samples) < min_samples_leaf and len(rightChild.samples) < min_samples_leaf:
            print('\033[1m' + f"pruning both children {leftChild.id} with {len(leftChild.samples)} samples and {rightChild.id} with {len(rightChild.samples)} samples" + '\033[0m')
            parentNode.leaf_node = True
            parentNode.left = None
            parentNode.right = None
        elif len(leftChild.samples) >= min_samples_leaf and len(rightChild.samples) >= min_samples_leaf:
            print("left turn to ", leftChild.id)
            pruneTree(leftChild, min_samples_leaf)
            print("right turn to ", rightChild.id)
            pruneTree(rightChild, min_samples_leaf)
        elif len(leftChild.samples) < min_samples_leaf:
            print('\033[1m' + f"pruning left child {leftChild.id} with {len(leftChild.samples)} samples" + '\033[0m')
            parentNode.left = None
            parentNode.threshold = -math.inf
            print("right turn to ", rightChild.id)
            pruneTree(rightChild, min_samples_leaf)
        elif len(rightChild.samples) < min_samples_leaf:
            print('\033[1m' + f"pruning right child {rightChild.id} with {len(rightChild.samples)} samples" + '\033[0m')
            parentNode.right = None
            parentNode.threshold = math.inf
            print("left turn to ", leftChild.id)
            pruneTree(leftChild, min_samples_leaf)

# Re-estimate node values and sample counts for subtree
def _reestimate_node_values(node):
    if node.leaf_node:
        return

    left_child = node.left
    right_child = node.right

    if not isinstance(node.samples, list):
        node.samples = [node.samples]  # Ensure samples is a list

    if node.threshold is not None:
        left_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] <= node.threshold]
        right_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] > node.threshold]

        if left_child:
            left_child.samples = left_samples
        if right_child:
            right_child.samples = right_samples

        if left_child:
            _reestimate_node_values(left_child)
        if right_child:
            _reestimate_node_values(right_child)

# Ensure initial tree creation uses list of indices
initial_samples = list(range(len(X_train)))
dt = deepcopy(rf.trees[0])
rootNode = dt.node_list[0]
rootNode.samples = initial_samples  # Ensure the root node starts with a list of all samples

# Prune and re-estimate
pruneTree(rootNode, 20)
print("------------ done pruning, now re-estimating:---------------")
_reestimate_node_values(rootNode)

# Traverse and verify
print("------------ Re-estimated tree: ---------------")
nLeafs = traverseTree(rootNode, 0)
print(nLeafs, "leaves in total")

inner node 0 with 75 samples and children: 1 2
pruning both children 1 with 1 samples and 2 with 1 samples
------------ done pruning, now re-estimating:---------------
------------ Re-estimated tree: ---------------
leaf node 0 with 75 samples
1 leaves in total


In [44]:
# Set a lower min_samples_leaf threshold if you want to retain more nodes
pruneTree(rootNode, min_samples_leaf=5)

leaf node 0 with 75 samples


In [45]:
# Prune tree function with correct sample redistribution
def pruneTree(parentNode, min_samples_leaf=10):
    if parentNode.leaf_node:
        print(f"Leaf node {parentNode.id} with {len(parentNode.samples)} samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right

        if not isinstance(parentNode.samples, list):
            raise ValueError(f"Node {parentNode.id} has samples as an integer, expected a list of samples.")

        # Ensure children nodes' samples are lists
        if leftChild is not None and isinstance(leftChild.samples, int):
            leftChild.samples = [leftChild.samples]
        if rightChild is not None and isinstance(rightChild.samples, int):
            rightChild.samples = [rightChild.samples]

        print(f"Inner node {parentNode.id} with {len(parentNode.samples)} samples and children: {leftChild.id}, {rightChild.id}")

        if len(leftChild.samples) < min_samples_leaf and len(rightChild.samples) < min_samples_leaf:
            print(f"Pruning both children {leftChild.id} with {len(leftChild.samples)} samples and {rightChild.id} with {len(rightChild.samples)} samples")

            # Redistribute samples of pruned nodes to the remaining subtree
            parentNode.leaf_node = True
            parentNode.left = None
            parentNode.right = None

        elif len(leftChild.samples) >= min_samples_leaf and len(rightChild.samples) >= min_samples_leaf:
            print(f"Left turn to {leftChild.id}")
            pruneTree(leftChild, min_samples_leaf)
            print(f"Right turn to {rightChild.id}")
            pruneTree(rightChild, min_samples_leaf)

        elif len(leftChild.samples) < min_samples_leaf:
            print(f"Pruning left child {leftChild.id} with {len(leftChild.samples)} samples")

            # Merge samples of pruned leftChild into rightChild
            rightChild.samples.extend(leftChild.samples)
            parentNode.left = None

            # Recursively prune the right subtree
            pruneTree(rightChild, min_samples_leaf)

        elif len(rightChild.samples) < min_samples_leaf:
            print(f"Pruning right child {rightChild.id} with {len(rightChild.samples)} samples")

            # Merge samples of pruned rightChild into leftChild
            leftChild.samples.extend(rightChild.samples)
            parentNode.right = None

            # Recursively prune the left subtree
            pruneTree(leftChild, min_samples_leaf)

# Re-estimate node values and sample counts for subtree
def _reestimate_node_values(node):
    if node.leaf_node:
        return

    left_child = node.left
    right_child = node.right

    if not isinstance(node.samples, list):
        node.samples = [node.samples]  # Ensure samples is a list

    if node.threshold is not None:
        left_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] <= node.threshold]
        right_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] > node.threshold]

        if left_child:
            left_child.samples = left_samples
        if right_child:
            right_child.samples = right_samples

        if left_child:
            _reestimate_node_values(left_child)
        if right_child:
            _reestimate_node_values(right_child)

# Usage
dt = deepcopy(rf.trees[0])
rootNode = dt.node_list[0]
rootNode.samples = initial_samples  # Ensure the root node starts with a list of all samples

# Prune and re-estimate
pruneTree(rootNode, 20)
print("------------ done pruning, now re-estimating:---------------")
_reestimate_node_values(rootNode)

# Traverse and verify
print("------------ Re-estimated tree: ---------------")
nLeafs = traverseTree(rootNode, 0)
print(nLeafs, "leaves in total")


Inner node 0 with 75 samples and children: 1, 2
Pruning both children 1 with 1 samples and 2 with 1 samples
------------ done pruning, now re-estimating:---------------
------------ Re-estimated tree: ---------------
leaf node 0 with 75 samples
1 leaves in total


In [50]:
import math

def ensure_samples_as_list(node):
    """
    Ensure that the samples attribute of a node is always a list.
    """
    if isinstance(node.samples, int):
        node.samples = [node.samples]
    elif isinstance(node.samples, list):
        pass  # Already a list
    else:
        raise TypeError(f"Unexpected type for samples in node {node.id}: {type(node.samples)}")

def redistribute_samples_down_the_tree(node, samples):
    """
    Recursively redistributes samples down the subtree rooted at `node`.
    """
    if node.leaf_node:
        node.samples.extend(samples)  # Correctly extend the list of samples
    else:
        left_samples = [sample for sample in samples if X_train.iloc[sample, 0] <= node.threshold]
        right_samples = [sample for sample in samples if X_train.iloc[sample, 0] > node.threshold]

        if node.left:
            redistribute_samples_down_the_tree(node.left, left_samples)
        if node.right:
            redistribute_samples_down_the_tree(node.right, right_samples)

def pruneTree(parentNode, min_samples_leaf=10):
    ensure_samples_as_list(parentNode)  # Ensure parentNode's samples are a list

    if parentNode.leaf_node:
        print(f"leaf node {parentNode.id} with {len(parentNode.samples)} samples")
        return
    else:
        leftChild = parentNode.left
        rightChild = parentNode.right

        if leftChild:
            ensure_samples_as_list(leftChild)
        if rightChild:
            ensure_samples_as_list(rightChild)

        print(f"inner node {parentNode.id} with {len(parentNode.samples)} samples and children: {leftChild.id if leftChild else None}, {rightChild.id if rightChild else None}")

        if leftChild and rightChild:
            if len(leftChild.samples) < min_samples_leaf and len(rightChild.samples) < min_samples_leaf:
                print(f'\033[1mPruning both children {leftChild.id} with {len(leftChild.samples)} samples and {rightChild.id} with {len(rightChild.samples)} samples\033[0m')
                parentNode.leaf_node = True
                parentNode.samples = list(set(parentNode.samples + leftChild.samples + rightChild.samples))  # Merge samples correctly
                parentNode.left = None
                parentNode.right = None
            elif len(leftChild.samples) >= min_samples_leaf and len(rightChild.samples) >= min_samples_leaf:
                print(f"left turn to {leftChild.id}")
                pruneTree(leftChild, min_samples_leaf)
                print(f"right turn to {rightChild.id}")
                pruneTree(rightChild, min_samples_leaf)
            elif len(leftChild.samples) < min_samples_leaf:
                print(f'\033[1mPruning left child {leftChild.id} with {len(leftChild.samples)} samples\033[0m')
                redistribute_samples_down_the_tree(rightChild, leftChild.samples)
                parentNode.left = None
                print(f"Removing useless inner node {parentNode.id} and promoting right child {rightChild.id}")
                parentNode.threshold = rightChild.threshold
                parentNode.samples = rightChild.samples
                parentNode.left = rightChild.left
                parentNode.right = rightChild.right
                parentNode.leaf_node = rightChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)
            elif len(rightChild.samples) < min_samples_leaf:
                print(f'\033[1mPruning right child {rightChild.id} with {len(rightChild.samples)} samples\033[0m')
                redistribute_samples_down_the_tree(leftChild, rightChild.samples)
                parentNode.right = None
                print(f"Removing useless inner node {parentNode.id} and promoting left child {leftChild.id}")
                parentNode.threshold = leftChild.threshold
                parentNode.samples = leftChild.samples
                parentNode.left = leftChild.left
                parentNode.right = leftChild.right
                parentNode.leaf_node = leftChild.leaf_node
                if not parentNode.leaf_node:
                    pruneTree(parentNode, min_samples_leaf)
        elif leftChild:
            if len(leftChild.samples) < min_samples_leaf:
                print(f'\033[1mPruning left child {leftChild.id} with {len(leftChild.samples)} samples\033[0m')
                parentNode.samples = list(set(parentNode.samples + leftChild.samples))  # Merge samples correctly
                parentNode.left = None
                parentNode.leaf_node = True
        elif rightChild:
            if len(rightChild.samples) < min_samples_leaf:
                print(f'\033[1mPruning right child {rightChild.id} with {len(rightChild.samples)} samples\033[0m')
                parentNode.samples = list(set(parentNode.samples + rightChild.samples))  # Merge samples correctly
                parentNode.right = None
                parentNode.leaf_node = True

def _reestimate_node_values(node):
    ensure_samples_as_list(node)

    if node.leaf_node:
        return

    left_child = node.left
    right_child = node.right

    left_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] <= node.threshold]
    right_samples = [sample for sample in node.samples if X_train.iloc[sample, 0] > node.threshold]

    if left_child:
        left_child.samples = left_samples
        _reestimate_node_values(left_child)
    if right_child:
        right_child.samples = right_samples
        _reestimate_node_values(right_child)

# Usage:
dt = deepcopy(rf.trees[0])
rootNode = dt.node_list[0]

# Initialize root samples as a list of indices
rootNode.samples = list(range(len(X_train)))

pruneTree(rootNode, 20)
print("------------ done pruning, now re-estimating:---------------")
_reestimate_node_values(rootNode)

# Traverse and verify
print("------------ Re-estimated tree: ---------------")
nLeafs = traverseTree(rootNode, 0)
print(f"{nLeafs} leaves in total")

inner node 0 with 75 samples and children: 1, 2
Pruning both children 1 with 1 samples and 2 with 1 samples
------------ done pruning, now re-estimating:---------------
------------ Re-estimated tree: ---------------
leaf node 0 with 75 samples
1 leaves in total
